In [1]:
!pip install regex pandarallel datasets torch transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 

In [15]:

import os

#os.environ['HF_HOME'] = '/data/users/ugarg/hf/hf_cache/'
#os.environ['TRANSFORMERS_CACHE'] = '/data/users/ugarg/hf/hf_cache/'
os.environ['CUDA_VISIBLE_DEVICES']='1'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pandarallel import pandarallel
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pandarallel.initialize(progress_bar=False)
from torch.nn.functional import one_hot


from collections import Counter

from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
import numpy as np
import random
from transformers import AdamW, AutoTokenizer,  AutoModel
from torch.nn.functional import one_hot
from collections import Counter
def seed_everything(seed: int):
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)
import joblib



INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [16]:
def format_table(df):
    df['labels'] = df['mr'].apply(lambda x: x.split('(',1)[0])
    df = df.drop('mr', 1)
    return df

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
train = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/DA Classifier/viggo-train.csv')
val = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/DA Classifier/viggo-valid.csv')
test = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/DA Classifier/viggo-test.csv')
train, test, valid = format_table(train), format_table(test), format_table(val)

<ipython-input-16-203a5f988e2c>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('mr', 1)
<ipython-input-16-203a5f988e2c>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('mr', 1)
<ipython-input-16-203a5f988e2c>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('mr', 1)


In [ ]:
# train.info()

In [ ]:
# train.labels.value_counts()

In [ ]:
# def show_sample(df, num_rows):
#     for i in range(num_rows):
#         idx = np.random.choice(len(df))
#         ref = df.loc[idx]['ref']
#         da = df.loc[idx]['labels']
#         #domain = df.loc[idx]['domain']
#         print('-'*50)
#         print(ref)
#         print(da)
#         #print(domain)
#         print('-'*50)

# print("\nSample Data: ")
# print(show_sample(train, 3))


In [10]:

DROPOUT = 0.3
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50
MAX_LENGTH = 100
BATCH_SIZE = 64
EARLY_STOPPING_LIMIT = 5
DEVICE = 'cpu'

save_checkpoint_path = '/content/drive/MyDrive/AA NLP244 Project Experiments/DA Classifier/Copy of best_model.pt'
resume_from_checkpoint = True




##################
#   tokenizer
##################


from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
print(f'\nLoading Tokenizer: {model_checkpoint} ...')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)




Loading Tokenizer: bert-base-cased ...


In [19]:


####################################
#   Create Pytorch Datasets
####################################

class TrainDataset:
    def __init__(self, train, tokenizer, max_length):
        self.train = train
        self.tokenizer = tokenizer
        self.source_column = 'ref'
        self.label_column = 'labels'
        self.max_length = max_length
        self.build_vocab()
    
    def __len__(self):
        return len(self.train)
    
    def build_vocab(self,):
         #get vocab
        self.stoi = {'request': 0,
                     'inform': 1,
                     'suggest': 2,
                     'request_attribute': 3,
                     'verify_attribute': 4,
                     'confirm': 5,
                     'give_opinion': 6,
                     'request_explanation': 7,
                     'recommend': 8}
        self.itos = {j:i for i,j in self.stoi.items()}
        
        self.num_class = len(self.itos)
        
        
    def __getitem__(self, idx):
        tokenized_source_text = self.tokenizer(self.train[self.source_column].loc[idx], truncation=True, padding=True, max_length = self.max_length)
        label = self.stoi[self.train[self.label_column][idx]]
        #one_hot_labels = one_hot(torch.tensor(label), num_classes = self.num_class)
        tokenized_source_text['label'] = label
        return tokenized_source_text

    
class ValDataset:
    def __init__(self, train_dataset, val, tokenizer, max_length):
        self.stoi = train_dataset.stoi
        self.itos = train_dataset.itos
        self.num_class = len(self.stoi)
        self.val = val
        self.tokenizer = tokenizer
        self.source_column = 'ref'
        self.label_column = 'labels'
        self.max_length = max_length
    def __len__(self):
        return len(self.val)

        
    def __getitem__(self, idx):
        tokenized_source_text = self.tokenizer(self.val[self.source_column].loc[idx], truncation=True, padding=True, max_length = self.max_length)
        label = self.stoi[self.val[self.label_column][idx]]
        tokenized_source_text['label'] = label
        return tokenized_source_text
        
        
print(f'\nCreating Pytorch Datasets and Loaders...')

print(f'\nKeeping max length: {MAX_LENGTH}\nBatch Size: {BATCH_SIZE}')




Creating Pytorch Datasets and Loaders...

Keeping max length: 100
Batch Size: 64


In [20]:

train_dataset = TrainDataset(train, tokenizer, max_length = MAX_LENGTH)
val_dataset = ValDataset(train_dataset, val, tokenizer, max_length = MAX_LENGTH)
test_dataset = ValDataset(train_dataset, test, tokenizer, max_length = MAX_LENGTH)


In [ ]:
# train_dataset.itos

In [ ]:
# train_dataset.stoi

In [21]:

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn = data_collator,
    batch_size=BATCH_SIZE,
)
val_dataloader = DataLoader(
    val_dataset, collate_fn = data_collator, batch_size=BATCH_SIZE
)

test_dataloader = DataLoader(
    test_dataset, collate_fn = data_collator, batch_size=BATCH_SIZE
)

print("\nShape of batch: ", next(iter(train_dataloader))['input_ids'].shape)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



Shape of batch:  torch.Size([64, 73])


In [22]:


####################################
#   Create Model Class
####################################


class Model(torch.nn.Module):
    def __init__(self, model_checkpoint, train_dataset, dropout, device):
        super().__init__()
        self.base = AutoModel.from_pretrained(
                model_checkpoint,
                #id2label=id2label,
                #label2id=label2id,
                )
        """
        Uncomment if want to use adapter
        #         self.base.add_adapter('adap')

        #         self.base.train_adapter("adap")
        #         self.base.set_active_adapters("adap")

        Uncomment if want to unfreeze specific layers
                # c = 0
                # for name, param in self.base.named_parameters():
                #     c+=1
                #     if c <= 100:
                #         param.requires_grad = False

        """
        self.stoi = train_dataset.stoi
        self.itos = train_dataset.itos
        self.device = device
        self.fc = torch.nn.Linear(self.base.config.hidden_size, len(self.stoi))
        
        self.dropout = torch.nn.Dropout(dropout)
        self.loss_fn = torch.nn.CrossEntropyLoss()    
        
        
        
    
    def forward(self, batch, mode = 'Train'):
        batch.to(self.device)
        self.base.to(self.device)
        self.fc.to(self.device)
        _ , pools = self.base(input_ids=batch['input_ids'],
                      attention_mask=batch['attention_mask'],
                      token_type_ids=batch['token_type_ids'],
                       return_dict=False)
        # print(hidden.shape)
        # print(pools.shape)

        if mode == 'Train':
            
            out = self.fc(self.dropout(pools))

            actual = batch['labels']
            loss = self.loss_fn(out, actual)

            return loss, out, actual
        else:
            
            out = self.fc(self.dropout(pools))
            return out




In [23]:

####################################
#   Load Model
####################################

print(f'\nUsing device {DEVICE} for training...')

model = Model(model_checkpoint, train_dataset, DROPOUT, DEVICE)
model.to(DEVICE)

####################################
#   Create Optimizer
####################################
print(f'\nCreating optimizer with Learning Rate: {LEARNING_RATE}')

from torch.optim import AdamW, Adam

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)


Using device cpu for training...


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Creating optimizer with Learning Rate: 0.0001


In [ ]:
# # In[8]:




# from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
# print(f'\nTraining for {NUM_EPOCHS} epochs with early stopping set to {EARLY_STOPPING_LIMIT}\n\n')

# num_train_epochs = NUM_EPOCHS
# num_update_steps_per_epoch = len(train_dataloader)
# num_training_steps = num_train_epochs * num_update_steps_per_epoch

# # lr_scheduler = get_scheduler(
# #     "linear",
# #     optimizer=optimizer,
# #     num_warmup_steps=500,
# #     num_training_steps=num_training_steps,
# # )
# lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
#     optimizer=optimizer,
#     num_warmup_steps=500,
#     num_training_steps=num_training_steps,
# )


# # In[9]:


# ####################################
# #   Create Checkpoint Builder
# ####################################

# def checkpoint_builder(model, optimizer, epoch, train_dataset, save_checkpoint_path):
#     """
#     desc:
#         creates checkpoint for the model
#     params:
#         model: trained model
#         optimizer
#         criterion: loss fn
#         epoch: checkpoint epoch
#         model_params: hyperparams of model
#         data_module: object of DataModule => contains the datasets and vocabulary
#         save_checkpoint_path: path to save checkpoint. Include checkpoint name
#     """
    
#     torch.save({'model_state_dict': model.state_dict(),
#                         'optim_state_dict':optimizer.state_dict(),
                        
#                         'epoch': epoch, 
                        
#                         'source_stoi': train_dataset.stoi,
#                         'source_itos': train_dataset.itos,
#                        }, save_checkpoint_path)


# # In[10]:




# from tqdm.auto import tqdm
# import torch

# progress_bar = tqdm(range(num_training_steps))
# early_stopping_counter = 0
# early_stopping_limit = EARLY_STOPPING_LIMIT
# best_valid_loss = float('inf')
# for epoch in range(num_train_epochs):
#     train_loss=0
#     valid_loss =0
    

    
#     print(f"[Epoch {epoch} / {num_train_epochs}]")
#     # Training
#     model.train()
#     for batch_idx, batch in enumerate(train_dataloader):
#         loss, out, actual = model(batch)
#         #loss = outputs.loss
#         loss.backward(loss)#, retain_graph = True)

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         train_loss+= ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
#         progress_bar.update(1)
#         progress_bar.set_postfix(loss = train_loss)
        
    
    
#     # Evaluation
#     model.eval()
#     for batch in val_dataloader:
#         with torch.no_grad():
#             loss, out, actual = model(batch)


#         labels = actual
#         predictions = out
        

#         valid_loss+= ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))
        

#     print('\nEpoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
#             epoch, 
#             train_loss,
#             valid_loss
#             ))

#     #early stopping, checkpointing
#     if valid_loss < best_valid_loss:
#         early_stopping_counter = 0
#         best_valid_loss = valid_loss
#         #create checkpoint
#         print("Loss improved saving checkpoint... ")
#         checkpoint_builder(model, optimizer, epoch, train_dataset, save_checkpoint_path)

#     else:
#         early_stopping_counter += 1
#         if early_stopping_counter >= early_stopping_limit:
#             print(f'\nLoss did not reduce for last {early_stopping_counter} epochs. Stopped training..')
#             break


# # In[ ]:





In [24]:


# ### Testing

# In[ ]:



if resume_from_checkpoint:
        print('Loading Checkpoint ...')
        checkpoint = torch.load(save_checkpoint_path, map_location=DEVICE)
        source_stoi = checkpoint['source_stoi']
        source_itos = checkpoint['source_itos']                
        print('Setting Models state to checkpoint...')
        #assert checkpoint['model_params'] == model_params
        model.load_state_dict(checkpoint['model_state_dict'])
        print('Model state set.')
        optimizer.load_state_dict(checkpoint['optim_state_dict'])
        print('Optimizer state set.')


Loading Checkpoint ...
Setting Models state to checkpoint...
Model state set.
Optimizer state set.


In [ ]:
# source_stoi

In [ ]:
# list(source_stoi.keys())

In [ ]:
# train_dataset.stoi

In [ ]:
# train_dataset[1]

In [ ]:




# #prints the examples that were wrongly predicted
# #prints the Accuracy and returns it
# #print the classification report
# def get_accuracy(dataloader, print_errors = False):
#     acc = []
#     preds = []
#     acts = []
#     for batch_idx, batch in enumerate(dataloader):
        
#         _, out, actual = model(batch)
#         out = torch.argmax(out, axis = 1)
#         out = out.cpu().numpy()
#         actual = actual.cpu().numpy()
        
#         acc.append([i == j for i, j in zip(out,actual)])
#         preds.append(out.tolist())
#         acts.append(actual.tolist())
#         del(out, actual,_,batch)
    
#     print(f'Accuracy: {np.mean(sum(acc,[]))}')
    
#     from sklearn.metrics import classification_report

#     label_names = list(source_stoi.keys())
#     print(classification_report(sum(acts,[]), sum(preds,[]),target_names=label_names))
#     from sklearn.metrics import confusion_matrix
#     from sklearn.metrics import ConfusionMatrixDisplay
#     dis = ConfusionMatrixDisplay(confusion_matrix(sum(acts,[]), sum(preds,[])), display_labels=label_names)
#     dis.plot(xticks_rotation=90)
#     plt.show()
    

# # In[15]:
# #model.eval()
# # model.to(DEVICE)
# print('\nValidation Results:')
# get_accuracy(val_dataloader, print_errors = True)
# print('\nTest Results:')
# get_accuracy(test_dataloader)








In [25]:
prompt_df = pd.read_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/NLP244_TV_chatgpt_90_responses.csv')

prompt_df = prompt_df[['mr','pseudo','callison', 'da', 'text' ]].dropna().reset_index(drop=True)

prompt_df.columns = ['mr','pseudo','callison', 'da', 'text']


In [27]:
def predict(col):
    df = prompt_df.copy()
    tokenized_data = tokenizer(col)
    tokenized_data = data_collator(tokenized_data)
    out = model(tokenized_data, mode = 'Test')
    probs = torch.nn.functional.softmax(out, dim = 1).detach().cpu().numpy()
    pred = out.argmax(dim = 1).cpu().numpy()
    pred = [train_dataset.itos[i] for i in pred]
    probs = [i.max() for i in probs]
    return pred, probs

#prompt_df = p8redict('candidate')
preds = []
probs = []
for i in range(len(prompt_df)): 
  print(i)
  pred, prob = predict([prompt_df["text"][i]])
  preds.extend(pred)
  probs.extend(prob)

prompt_df["pred"] = preds
prompt_df["probs"] = probs

prompt_df.to_csv('/content/drive/MyDrive/AA NLP244 Project Experiments/Evaluation/ChatGPT/NLP244_TV_chatgpt_eval-DA.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [ ]:
from datasets import Dataset as D
def tokenize(values): 
   input_ids = tokenizer(values["text"])
   return input_ids

def predict(df, col):
    dataset = D.from_pandas(df[])
    #print(dataset)
    tokenized_data = dataset.map(tokenize, batched=True, batch_size=64)
    # print(type(tokenized_data)) 
    # print(tokenized_data)
    dicts = {"input_ids":tokenized_data["input_ids"], "token_type_ids":tokenized_data["token_type_ids"]}
    tokenized_data = data_collator(dicts)
    out = model(tokenized_data, mode = 'Test')
    probs = torch.nn.functional.softmax(out, dim = 1).detach().cpu().numpy()
    pred = out.argmax(dim = 1).cpu().numpy()
    pred = [train_dataset.itos[i] for i in pred]
    probs = [i.max() for i in probs]

    # df[f'{col}_pred'] = pred
    # df[f'{col}_prob'] = probs
    # for i in range(len(df)):
    #     df.loc[i,f'{col}_pred'] = pred[i]
    #     df.loc[i,f'{col}_prob'] = probs[i]
    return pred, probs

#prompt_df = p8redict('candidate')
preds = []
probs = []
for i in range(len(prompt_df)): 
  print(prompt_df["text"][i])
  pred, prob = predict(prompt_df["text"][i])
  preds.append(pred)
  probs.append(prob)

df["pred"] = preds
df["probs"] = probs
#prompt_df['actual'] = prompt_df['mr'].apply(lambda x: x.split('|')[0].split('=')[0].strip())

#print(prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['da'] else 0,axis=1).value_counts())

prompt_df.to_csv('/content/drive/MyDrive/KG-NLG Jurassic/Experiments NLP 244/kg_outputs/jurassic_eval-DA.csv')

KeyError: ignored

In [ ]:
prompt_df = pd.read_csv('/content/drive/MyDrive/KG-NLG Jurassic/Experiments NLP 244/kg_outputs/chatgpt_eval - chatgpt_eval.csv')

prompt_df = prompt_df[['index', 'mr','pseudo','callison', 'da', 'text' ]].dropna().reset_index(drop=True)

prompt_df.columns = ['index', 'mr','pseudo','callison', 'da', 'text']

def predict(col):
    df = prompt_df.copy()
    tokenized_data = tokenizer(df[col].values.tolist())
    print(tokenized_data)
    print(type(tokenized_data))
    tokenized_data = data_collator(tokenized_data)
    print(type(tokenized_data))
    out = model(tokenized_data, mode = 'Test')
    probs = torch.nn.functional.softmax(out, dim = 1).detach().cpu().numpy()
    pred = out.argmax(dim = 1).cpu().numpy()
    pred = [train_dataset.itos[i] for i in pred]
    probs = [i.max() for i in probs]

    df[f'{col}_pred'] = 1
    df[f'{col}_prob'] = 1
    for i in range(len(df)):
        df.loc[i,f'{col}_pred'] = pred[i]
        df.loc[i,f'{col}_prob'] = probs[i]
    return df

#prompt_df = predict('candidate')
prompt_df = predict('text')
#prompt_df['actual'] = prompt_df['mr'].apply(lambda x: x.split('|')[0].split('=')[0].strip())

#print(prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['da'] else 0,axis=1).value_counts())

prompt_df.to_csv('/content/drive/MyDrive/KG-NLG Jurassic/Experiments NLP 244/kg_outputs/chatgpt_eval-DA.csv')

In [ ]:
# prompt_df['check'] = prompt_df.candidate.apply(lambda x: 1 if 'did' in x else 0)

In [ ]:
# prompt_df[prompt_df.check==1]

In [ ]:
# prompt_df = pd.read_csv('../test_files/viggo-test-specific-10examples-new-format-1-100 - viggo-test-specific-10examples-1-100.csv')

# prompt_df = prompt_df[['MR', 'Candidate','best ref',' DA (perfect  3, 2, 1 means not really at all)']].dropna().reset_index(drop=True)

# prompt_df.columns = ['mr', 'candidate', 'best_ref', 'related']

# def predict(col):
#     df = prompt_df.copy()
#     tokenized_data = tokenizer(df[col].values.tolist())
#     tokenized_data = data_collator(tokenized_data)

#     out = model(tokenized_data, mode = 'Test')
#     probs = torch.nn.functional.softmax(out, dim = 1).detach().cpu().numpy()
#     pred = out.argmax(dim = 1).cpu().numpy()
#     pred = [train_dataset.itos[i] for i in pred]
#     probs = [i.max() for i in probs]

#     df[f'{col}_pred'] = 1
#     df[f'{col}_prob'] = 1
#     for i in range(len(df)):
#         df.loc[i,f'{col}_pred'] = pred[i]
#         df.loc[i,f'{col}_prob'] = probs[i]
#     return df

# prompt_df = predict('candidate')
# prompt_df = predict('best_ref')
# prompt_df['actual'] = prompt_df['mr'].apply(lambda x: x.split('|')[0].split('=')[0].split(':')[1].strip())
# print(prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['actual'] else 0,axis=1).value_counts())
# prompt_df.to_csv('viggo-test-specific-10examples-new-format-1-100 - viggo-test-specific-10examples-1-100.csv')

In [ ]:
# prompt_df.related.value_counts()

In [ ]:
# print(prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['actual'] else 0,axis=1).value_counts())

In [ ]:
# print(prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['actual'] and x['related']!=3 else 0,axis=1).value_counts())

In [ ]:
# prompt_df = pd.read_csv('../test_files/viggo_test_5_shuffle_output - cc-clean.csv')

# prompt_df = prompt_df[['mr', 'generated']].reset_index(drop=True)

# prompt_df.columns = ['mr', 'candidate']

# def predict(col):
#     df = prompt_df.copy()
#     tokenized_data = tokenizer(df[col].values.tolist())
#     tokenized_data = data_collator(tokenized_data)

#     out = model(tokenized_data, mode = 'Test')
#     probs = torch.nn.functional.softmax(out, dim = 1).detach().cpu().numpy()
#     pred = out.argmax(dim = 1).cpu().numpy()
#     pred = [train_dataset.itos[i] for i in pred]
#     probs = [i.max() for i in probs]

#     df[f'{col}_pred'] = 1
#     df[f'{col}_prob'] = 1
#     for i in range(len(df)):
#         df.loc[i,f'{col}_pred'] = pred[i]
#         df.loc[i,f'{col}_prob'] = probs[i]
#     return df

# prompt_df = predict('candidate')
# #prompt_df = predict('best_ref')
# prompt_df['actual'] = prompt_df['mr'].apply(lambda x: x.split('|')[0].split('=')[0].strip())
# print(prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['actual'] else 0,axis=1).value_counts())
# prompt_df.to_csv('viggo_test_5_shuffle_output - cc-clean.csv')

In [ ]:
# prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['actual'] else 0, axis=1).value_counts()

In [ ]:
# prompt_df[['mr', 'candidate_pred', 'candidate_prob']].to_csv('viggo_test_5_shuffle_output - cc-clean.csv')

In [ ]:
# prompt_df = pd.read_csv('../test_files/viggo_test_5_shuffle_output - cc-clean.csv')

In [ ]:
# prompt_df[prompt_df.candidate=='Do you like playing Diablo II?']

In [ ]:
# prompt_df = pd.read_csv('../test_files/viggo_test_custom25_output - viggo_test_custom_output.csv')


In [ ]:
# prompt_df[['mr', 'generated']].info()

In [ ]:
# prompt_df = pd.read_csv('../test_files/viggo_test_custom25_output - viggo_test_custom_output.csv')

# prompt_df = prompt_df[['mr','generated']].dropna().reset_index(drop=True)



# def predict(col):
#     df = prompt_df.copy()
#     tokenized_data = tokenizer(df[col].values.tolist())
#     tokenized_data = data_collator(tokenized_data)
    
    
#     out = model(tokenized_data, mode = 'Test')
#     probs = torch.nn.functional.softmax(out, dim = 1).detach().cpu().numpy()
#     pred = out.argmax(dim = 1).cpu().numpy()
#     pred = [train_dataset.itos[i] for i in pred]
#     probs = [i.max() for i in probs]

#     df[f'{col}_pred'] = 1
#     df[f'{col}_prob'] = 1
#     for i in range(len(df)):
#         df.loc[i,f'{col}_pred'] = pred[i]
#         df.loc[i,f'{col}_prob'] = probs[i]
#     return df

# prompt_df = predict('generated')
# #prompt_df = predict('best_ref')
# prompt_df['actual'] = prompt_df['mr'].apply(lambda x: x.split('|')[0].split('=')[0].strip())

# print(prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['actual'] else 0,axis=1).value_counts())

# prompt_df.to_csv('../test_files/viggo_test_custom25_output - viggo_test_custom_output.csv')

In [ ]:
prompt_df = pd.read_csv('../test_files/viggo_test_custom25_output - viggo_test_custom_output.csv')

prompt_df = prompt_df[['mr','generated']].dropna().reset_index(drop=True)



def predict(col):
    df = prompt_df.copy()
    tokenized_data = tokenizer(df[col].values.tolist())
    tokenized_data = data_collator(tokenized_data)
    print(tokenized_data)
    outs = []
    
    for i in range(0,len(tokenized_data['input_ids']), 100):
        out = model(tokenized_data['input_ids'][i:i+100], mode = 'Test')
        outs.append(out)
    
    
    probs = torch.nn.functional.softmax(out, dim = 1).detach().cpu().numpy()
    pred = out.argmax(dim = 1).cpu().numpy()
    pred = [train_dataset.itos[i] for i in pred]
    probs = [i.max() for i in probs]

    df[f'{col}_pred'] = 1
    df[f'{col}_prob'] = 1
    for i in range(len(df)):
        df.loc[i,f'{col}_pred'] = pred[i]
        df.loc[i,f'{col}_prob'] = probs[i]
    return df

prompt_df = predict('generated')
#prompt_df = predict('best_ref')
prompt_df['actual'] = prompt_df['mr'].apply(lambda x: x.split('|')[0].split('=')[0].strip())

print(prompt_df.apply(lambda x: 1 if x['candidate_pred'] == x['actual'] else 0,axis=1).value_counts())

prompt_df.to_csv('../test_files/viggo_test_custom25_output - viggo_test_custom_output.csv')